# Old Combiner


- Eliminates Duplicates without respect to Service Type

In [2]:
import pandas as pd
import glob
import os
import logging
from datetime import datetime

def merge_csv_files(input_directory, output_filename="merged_output.csv", 
                   id_column=None, keep='first', log_level=logging.INFO):
    """
    Merge multiple CSV files from a directory into a single CSV file,
    removing duplicates based on ACN or client ID column.
    
    Args:
        input_directory (str): Path to directory containing CSV files
        output_filename (str): Name of the output merged CSV file (default: merged_output.csv)
        id_column (str): Column name to use for identifying unique records (ACN or client ID)
                         If None, will try to find "ACN" or "client id" automatically
        keep (str): Which duplicates to keep {'first', 'last', False}
                    - 'first': Keep first occurrence of duplicates
                    - 'last': Keep last occurrence of duplicates
                    - False: Remove all duplicates
                    (default: 'first')
        log_level (int): Logging level (default: logging.INFO)
    
    Returns:
        tuple: (output_path, stats_dict) where stats_dict contains detailed statistics
    """
    # Set up logging
    log_filename = f"csv_merge_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    log_path = os.path.join(input_directory, log_filename)
    
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_path),
            logging.StreamHandler()
        ]
    )
    
    logger = logging.getLogger("csv_merger")
    logger.info(f"Starting CSV merge operation in directory: {input_directory}")
    
    # Get all CSV files in the directory
    csv_files = glob.glob(os.path.join(input_directory, "*.csv"))
    
    if not csv_files:
        logger.error(f"No CSV files found in {input_directory}")
        raise ValueError(f"No CSV files found in {input_directory}")
    
    logger.info(f"Found {len(csv_files)} CSV files to process")
    
    # Read and combine all CSV files
    dfs = []
    file_record_counts = {}
    column_names_by_file = {}
    
    for file in csv_files:
        filename = os.path.basename(file)
        try:
            df = pd.read_csv(file)
            rows = len(df)
            file_record_counts[filename] = rows
            column_names_by_file[filename] = list(df.columns)
            dfs.append(df)
            logger.info(f"Read {rows} records from {filename}")
        except Exception as e:
            logger.error(f"Error reading {filename}: {str(e)}")
            continue
    
    if not dfs:
        logger.error("No data was read from any CSV files")
        raise ValueError("No data was read from any CSV files")
    
    # Concatenate all dataframes
    merged_df = pd.concat(dfs, ignore_index=True)
    total_records = len(merged_df)
    logger.info(f"Total records after concatenation: {total_records}")
    
    # Determine the ID column if not specified
    if id_column is None:
        # Look for common ID column names (case-insensitive)
        possible_id_columns = [col for col in merged_df.columns 
                             if col.upper() == "ACN" or col.lower() == "client id" 
                             or col.lower() == "clientid" or col.lower() == "client_id"
                             or col == "User ID"]
        
        if possible_id_columns:
            id_column = possible_id_columns[0]
            logger.info(f"Automatically selected '{id_column}' as the ID column")
        else:
            logger.error("Could not automatically determine ID column. Please specify 'id_column'.")
            raise ValueError("Could not automatically determine ID column. Please specify 'id_column'.")
    
    # Validate that ID column exists
    if id_column not in merged_df.columns:
        logger.error(f"ID column '{id_column}' not found in the data. Available columns: {list(merged_df.columns)}")
        raise ValueError(f"ID column '{id_column}' not found in the data")
    
    # Check for missing values in ID column
    missing_ids = merged_df[id_column].isna().sum()
    if missing_ids > 0:
        logger.warning(f"Found {missing_ids} records with missing values in ID column '{id_column}'")
    
    # Count unique IDs before deduplication
    unique_ids_before = merged_df[id_column].nunique()
    logger.info(f"Found {unique_ids_before} unique {id_column} values before deduplication")
    
    # Create detailed statistics
    stats = {
        "original_record_count": total_records,
        "file_record_counts": file_record_counts,
        "column_names_by_file": column_names_by_file,
        "unique_ids_before": unique_ids_before,
        "id_column_used": id_column,
        "records_with_missing_ids": missing_ids
    }
    
    # Find duplicates based on ID column
    duplicates = merged_df.duplicated(subset=[id_column], keep=False)
    duplicate_count = duplicates.sum()
    duplicate_records = merged_df[duplicates].copy()
    
    # Log duplicate statistics
    stats["total_duplicate_records"] = duplicate_count
    stats["duplicate_ids_count"] = duplicate_records[id_column].nunique()
    
    if duplicate_count > 0:
        logger.info(f"Found {duplicate_count} duplicate records based on {id_column}")
        logger.info(f"These duplicates belong to {stats['duplicate_ids_count']} unique {id_column} values")
        
        # Save duplicates to a separate file for review
        duplicates_path = os.path.join(input_directory, f"duplicates_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
        duplicate_records.to_csv(duplicates_path, index=False)
        logger.info(f"Saved duplicate records to {duplicates_path} for review")
        
        # Log top 5 IDs with most duplicates
        duplicate_counts = duplicate_records[id_column].value_counts()
        top_duplicates = duplicate_counts.head(5)
        logger.info(f"Top {len(top_duplicates)} IDs with most duplicates:")
        for id_val, count in top_duplicates.items():
            logger.info(f"  {id_column}: {id_val}, Count: {count}")
        
        stats["top_duplicate_ids"] = {str(id_val): int(count) for id_val, count in top_duplicates.items()}
    
    # Remove duplicates based on ID column
    original_count = len(merged_df)
    merged_df = merged_df.drop_duplicates(subset=[id_column], keep=keep)
    removed_count = original_count - len(merged_df)
    
    # Update statistics
    stats["duplicate_records_removed"] = removed_count
    stats["final_record_count"] = len(merged_df)
    stats["unique_ids_after"] = merged_df[id_column].nunique()
    
    logger.info(f"Removed {removed_count} duplicate records based on {id_column}")
    logger.info(f"Final record count: {stats['final_record_count']}")
    logger.info(f"Final unique {id_column} count: {stats['unique_ids_after']}")
    
    # Create output path in the same directory
    output_path = os.path.join(input_directory, output_filename)
    
    # Save to output file
    merged_df.to_csv(output_path, index=False)
    logger.info(f"Successfully merged files into {output_path}")
    
    # Create a simple stats report file
    stats_path = os.path.join(input_directory, f"merge_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
    with open(stats_path, 'w') as f:
        f.write(f"CSV Merge Operation Statistics - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Input Directory: {input_directory}\n")
        f.write(f"Output File: {output_filename}\n")
        f.write(f"ID Column Used: {id_column}\n\n")
        
        f.write("Summary Statistics:\n")
        f.write(f"  Total files processed: {len(csv_files)}\n")
        f.write(f"  Total records: {total_records}\n")
        f.write(f"  Unique {id_column} values before deduplication: {unique_ids_before}\n")
        f.write(f"  Duplicate records found: {duplicate_count}\n")
        f.write(f"  Records removed: {removed_count}\n")
        f.write(f"  Final record count: {stats['final_record_count']}\n")
        f.write(f"  Final unique {id_column} count: {stats['unique_ids_after']}\n\n")
        
        f.write("File Details:\n")
        for filename, count in file_record_counts.items():
            f.write(f"  {filename}: {count} records\n")
    
    logger.info(f"Saved detailed statistics to {stats_path}")
    logger.info("CSV merge operation completed successfully")
    
    return output_path, stats

# Example usage
if __name__ == "__main__":
    # Replace with your input directory
    input_dir = "/Users/byron/Downloads"
    
    # Use specific column for identifying duplicates
    output_path, stats = merge_csv_files(
        input_directory=input_dir, 
        output_filename="merged_unique_clients.csv",
        id_column="User ID",  # Specifically use "User ID" column
        keep="first",    # Keep first occurrence of each duplicate
        log_level=logging.INFO
    )
    
    # Example: Alternative ways to use the function
    
    # Example 1: Specify the exact ID column name
    # output_path, stats = merge_csv_files(
    #     input_directory=input_dir,
    #     output_filename="merged_unique_acn.csv",
    #     id_column="ACN",
    #     keep="first"
    # )
    
    # Example 2: Keep last occurrence of each duplicate instead
    # output_path, stats = merge_csv_files(
    #     input_directory=input_dir,
    #     output_filename="merged_unique_latest.csv",
    #     id_column="client id",
    #     keep="last"
    # )
    
    # Example 3: Use debug level logging for more detailed output
    # output_path, stats = merge_csv_files(
    #     input_directory=input_dir,
    #     log_level=logging.DEBUG
    # )

2025-05-29 14:57:37,016 - INFO - Starting CSV merge operation in directory: /Users/byron/Downloads
2025-05-29 14:57:37,017 - INFO - Found 15 CSV files to process
2025-05-29 14:57:37,020 - INFO - Read 50 records from client_data_2025-05-29 (2).csv
2025-05-29 14:57:37,022 - INFO - Read 47 records from client_data_2025-05-29 (12).csv
2025-05-29 14:57:37,024 - INFO - Read 47 records from client_data_2025-05-29 (13).csv
2025-05-29 14:57:37,026 - INFO - Read 49 records from client_data_2025-05-29 (3).csv
2025-05-29 14:57:37,027 - INFO - Read 29 records from client_data_2025-05-29 (14).csv
2025-05-29 14:57:37,033 - INFO - Read 45 records from client_data_2025-05-29 (8).csv
2025-05-29 14:57:37,040 - INFO - Read 47 records from client_data_2025-05-29.csv
2025-05-29 14:57:37,048 - INFO - Read 49 records from client_data_2025-05-29 (4).csv
2025-05-29 14:57:37,056 - INFO - Read 47 records from client_data_2025-05-29 (5).csv
2025-05-29 14:57:37,058 - INFO - Read 48 records from client_data_2025-05-

# New Combiner


- Based new Extractor that takes TYPE from service Items
- Sorts based on Type and then eliminates duplicates within groups not on the whole dataset
- Used for simplified adding to DEX Client Data

In [3]:
import pandas as pd
import glob
import os
import logging
from datetime import datetime

def merge_csv_files(input_directory, output_filename="merged_output.csv", 
                   id_column=None, type_column="Type", keep='last', log_level=logging.INFO):
    """
    Merge multiple CSV files from a directory into a single CSV file,
    removing duplicates based on ACN or client ID column, but only within same type group.
    
    Args:
        input_directory (str): Path to directory containing CSV files
        output_filename (str): Name of the output merged CSV file (default: merged_output.csv)
        id_column (str): Column name to use for identifying unique records (ACN or client ID)
                         If None, will try to find "ACN" or "client id" automatically
        type_column (str): Column name for the service type (default: "Type")
        keep (str): Which duplicates to keep {'first', 'last', False}
                    - 'first': Keep first occurrence of duplicates
                    - 'last': Keep last occurrence of duplicates
                    - False: Remove all duplicates
                    (default: 'first')
        log_level (int): Logging level (default: logging.INFO)
    
    Returns:
        tuple: (output_path, stats_dict) where stats_dict contains detailed statistics
    """
    # Set up logging
    log_filename = f"csv_merge_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    log_path = os.path.join(input_directory, log_filename)
    
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_path),
            logging.StreamHandler()
        ]
    )
    
    logger = logging.getLogger("csv_merger")
    logger.info(f"Starting CSV merge operation in directory: {input_directory}")
    
    # Get all CSV files in the directory
    csv_files = glob.glob(os.path.join(input_directory, "*.csv"))
    
    if not csv_files:
        logger.error(f"No CSV files found in {input_directory}")
        raise ValueError(f"No CSV files found in {input_directory}")
    
    logger.info(f"Found {len(csv_files)} CSV files to process")
    
    # Read and combine all CSV files
    dfs = []
    file_record_counts = {}
    column_names_by_file = {}
    
    for file in csv_files:
        filename = os.path.basename(file)
        try:
            df = pd.read_csv(file)
            rows = len(df)
            file_record_counts[filename] = rows
            column_names_by_file[filename] = list(df.columns)
            dfs.append(df)
            logger.info(f"Read {rows} records from {filename}")
        except Exception as e:
            logger.error(f"Error reading {filename}: {str(e)}")
            continue
    
    if not dfs:
        logger.error("No data was read from any CSV files")
        raise ValueError("No data was read from any CSV files")
    
    # Concatenate all dataframes
    merged_df = pd.concat(dfs, ignore_index=True)
    total_records = len(merged_df)
    logger.info(f"Total records after concatenation: {total_records}")
    
    # Determine the ID column if not specified
    if id_column is None:
        # Look for common ID column names (case-insensitive)
        possible_id_columns = [col for col in merged_df.columns 
                             if col.upper() == "ACN" or col.lower() == "client id" 
                             or col.lower() == "clientid" or col.lower() == "client_id"
                             or col == "User ID"]
        
        if possible_id_columns:
            id_column = possible_id_columns[0]
            logger.info(f"Automatically selected '{id_column}' as the ID column")
        else:
            logger.error("Could not automatically determine ID column. Please specify 'id_column'.")
            raise ValueError("Could not automatically determine ID column. Please specify 'id_column'.")
    
    # Validate that ID column exists
    if id_column not in merged_df.columns:
        logger.error(f"ID column '{id_column}' not found in the data. Available columns: {list(merged_df.columns)}")
        raise ValueError(f"ID column '{id_column}' not found in the data")
    
    # Validate that Type column exists
    if type_column not in merged_df.columns:
        logger.error(f"Type column '{type_column}' not found in the data. Available columns: {list(merged_df.columns)}")
        raise ValueError(f"Type column '{type_column}' not found in the data")
    
    # Check for missing values in ID column
    missing_ids = merged_df[id_column].isna().sum()
    if missing_ids > 0:
        logger.warning(f"Found {missing_ids} records with missing values in ID column '{id_column}'")
    
    # Count unique IDs before deduplication
    unique_ids_before = merged_df[id_column].nunique()
    logger.info(f"Found {unique_ids_before} unique {id_column} values before deduplication")
    
    # Sort by Type (HM, DA) - ensure HM types come first
    merged_df['sort_order'] = merged_df[type_column].apply(lambda x: 0 if x == 'HM' else 1 if x == 'DA' else 2)
    merged_df.sort_values(by=['sort_order', id_column], inplace=True)
    merged_df.drop('sort_order', axis=1, inplace=True)
    
    logger.info(f"Sorted data by {type_column} (HM first, then DA)")
    
    # Create detailed statistics
    stats = {
        "original_record_count": total_records,
        "file_record_counts": file_record_counts,
        "column_names_by_file": column_names_by_file,
        "unique_ids_before": unique_ids_before,
        "id_column_used": id_column,
        "type_column_used": type_column,
        "records_with_missing_ids": missing_ids
    }
    
    # Group by type and check for duplicates within each type
    type_groups = merged_df[type_column].unique()
    logger.info(f"Found {len(type_groups)} distinct service types: {', '.join(map(str, type_groups))}")
    
    all_duplicates = pd.DataFrame()
    total_dupes_removed = 0
    
    # Process each type group separately
    deduped_dfs = []
    for service_type in type_groups:
        type_df = merged_df[merged_df[type_column] == service_type].copy()
        type_count_before = len(type_df)
        
        # Find duplicates based on ID column within this type
        type_duplicates = type_df.duplicated(subset=[id_column], keep=False)
        type_duplicate_count = type_duplicates.sum()
        
        if type_duplicate_count > 0:
            duplicate_records = type_df[type_duplicates].copy()
            all_duplicates = pd.concat([all_duplicates, duplicate_records])
            
            # Count of unique IDs with duplicates in this type
            duplicate_ids_count = duplicate_records[id_column].nunique()
            logger.info(f"In {service_type}: Found {type_duplicate_count} duplicate records for {duplicate_ids_count} unique {id_column} values")
            
            # Remove duplicates
            type_df = type_df.drop_duplicates(subset=[id_column], keep=keep)
            removed_count = type_count_before - len(type_df)
            total_dupes_removed += removed_count
            
            logger.info(f"In {service_type}: Removed {removed_count} duplicate records")
        else:
            logger.info(f"In {service_type}: No duplicates found")
        
        deduped_dfs.append(type_df)
    
    # Combine the deduplicated dataframes
    merged_df = pd.concat(deduped_dfs, ignore_index=True)
    
    # Save duplicates to a separate file if any were found
    if not all_duplicates.empty:
        duplicates_path = os.path.join(input_directory, f"duplicates_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
        all_duplicates.to_csv(duplicates_path, index=False)
        logger.info(f"Saved duplicate records to {duplicates_path} for review")
    
    # Update statistics
    stats["duplicate_records_removed"] = total_dupes_removed
    stats["final_record_count"] = len(merged_df)
    stats["unique_ids_after"] = merged_df[id_column].nunique()
    
    # Count records by type
    type_counts = merged_df[type_column].value_counts().to_dict()
    stats["records_by_type"] = type_counts
    
    logger.info(f"Final record count: {stats['final_record_count']}")
    logger.info(f"Final unique {id_column} count: {stats['unique_ids_after']}")
    for type_val, count in type_counts.items():
        logger.info(f"Service type {type_val}: {count} records")
    
    # Count clients with both service types
    if 'HM' in type_counts and 'DA' in type_counts:
        hm_clients = set(merged_df[merged_df[type_column] == 'HM'][id_column])
        da_clients = set(merged_df[merged_df[type_column] == 'DA'][id_column])
        clients_with_both = hm_clients.intersection(da_clients)
        
        stats["clients_with_both_service_types"] = len(clients_with_both)
        logger.info(f"Clients with both HM and DA services: {len(clients_with_both)}")
    
    # Create output path in the same directory
    output_path = os.path.join(input_directory, output_filename)
    
    # Save to output file
    merged_df.to_csv(output_path, index=False)
    logger.info(f"Successfully merged files into {output_path}")
    
    # Create a simple stats report file
    stats_path = os.path.join(input_directory, f"merge_stats_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
    with open(stats_path, 'w') as f:
        f.write(f"CSV Merge Operation Statistics - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Input Directory: {input_directory}\n")
        f.write(f"Output File: {output_filename}\n")
        f.write(f"ID Column Used: {id_column}\n")
        f.write(f"Type Column Used: {type_column}\n\n")
        
        f.write("Summary Statistics:\n")
        f.write(f"  Total files processed: {len(csv_files)}\n")
        f.write(f"  Total records: {total_records}\n")
        f.write(f"  Unique {id_column} values before deduplication: {unique_ids_before}\n")
        f.write(f"  Duplicate records removed: {total_dupes_removed}\n")
        f.write(f"  Final record count: {stats['final_record_count']}\n")
        f.write(f"  Final unique {id_column} count: {stats['unique_ids_after']}\n\n")
        
        f.write("Records by Type:\n")
        for type_val, count in type_counts.items():
            f.write(f"  {type_val}: {count} records\n")
        
        if 'clients_with_both_service_types' in stats:
            f.write(f"\nClients with both HM and DA services: {stats['clients_with_both_service_types']}\n\n")
        
        f.write("File Details:\n")
        for filename, count in file_record_counts.items():
            f.write(f"  {filename}: {count} records\n")
    
    logger.info(f"Saved detailed statistics to {stats_path}")
    logger.info("CSV merge operation completed successfully")
    
    return output_path, stats

# Example usage
if __name__ == "__main__":
    # Replace with your input directory
    input_dir = "/Users/byron/Downloads"
    
	
    # Use specific column for identifying duplicates
    output_path, stats = merge_csv_files(
        input_directory=input_dir, 
        output_filename="merged_unique_clients.csv",
        id_column="ACN",    # Specifically use "User ID" column
        type_column="Type",     # Service type column
        keep="first",           # Keep first occurrence of each duplicate within type
        log_level=logging.INFO
    )

2025-05-29 16:42:38,970 - INFO - Starting CSV merge operation in directory: /Users/byron/Downloads
2025-05-29 16:42:38,971 - INFO - Found 16 CSV files to process
2025-05-29 16:42:38,973 - INFO - Read 47 records from client_data_2025-05-29 (2).csv
2025-05-29 16:42:38,975 - INFO - Read 49 records from client_data_2025-05-29 (12).csv
2025-05-29 16:42:38,977 - INFO - Read 49 records from client_data_2025-05-29 (13).csv


2025-05-29 16:42:38,979 - INFO - Read 44 records from client_data_2025-05-29 (3).csv
2025-05-29 16:42:38,986 - INFO - Read 47 records from client_data_2025-05-29 (14).csv
2025-05-29 16:42:38,998 - INFO - Read 47 records from client_data_2025-05-29 (8).csv
2025-05-29 16:42:39,006 - INFO - Read 42 records from client_data_2025-05-29.csv
2025-05-29 16:42:39,008 - INFO - Read 48 records from client_data_2025-05-29 (4).csv
2025-05-29 16:42:39,011 - INFO - Read 50 records from client_data_2025-05-29 (5).csv
2025-05-29 16:42:39,013 - INFO - Read 47 records from client_data_2025-05-29 (9).csv
2025-05-29 16:42:39,015 - INFO - Read 25 records from client_data_2025-05-29 (15).csv
2025-05-29 16:42:39,016 - INFO - Read 45 records from client_data_2025-05-29 (6).csv
2025-05-29 16:42:39,018 - INFO - Read 48 records from client_data_2025-05-29 (7).csv
2025-05-29 16:42:39,020 - INFO - Read 49 records from client_data_2025-05-29 (10).csv
2025-05-29 16:42:39,021 - INFO - Read 50 records from client_data_